In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
source_folder = '/content/drive/My Drive/Sihem/BERT_Classifier/Data'
destination_folder = '/content/drive/My Drive/Sihem/BERT_Classifier/Model_afterPreprocessing'

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
model_name = "bert-base-uncased"


In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)


In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name)


In [ ]:
df = pd.read_csv(source_folder+"/train.csv")
df_val = pd.read_csv(source_folder+"/valid.csv")
df_test = pd.read_csv(source_folder+"/test.csv")

In [ ]:
df = df.dropna()
df_val = df_val.dropna()
df_test = df_test.dropna()
len(df)

In [ ]:
#Data preprocessing
import re
import string
def text_preproc(x):
  #x = x.lower()
  #x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x
df['text'] = df.text.apply(text_preproc)
df_val['text'] = df_val.text.apply(text_preproc)
df_test['text'] = df_test.text.apply(text_preproc)

In [ ]:
df.columns

In [ ]:
df.tail()

In [ ]:
def round_val(val):
    return round(val)

In [ ]:

label_name = 'label'
threshold = 0
df = df[(df[label_name] >= 1-threshold)|(df[label_name] <=threshold)]
df_val = df_val[(df_val[label_name] >= 1-threshold)|(df_val[label_name] <=threshold)]
df_test = df_test[(df_test[label_name] >= 1-threshold) | (df_test[label_name] <=threshold)]

df[label_name] = df[label_name].apply(round_val)
df_val[label_name] = df_val[label_name].apply(round_val)
df_test[label_name] = df_test[label_name].apply(round_val)

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)
df_val = shuffle(df_val)
df_test = shuffle(df_test)
df.reset_index(inplace=True, drop=True)
df_val.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [ ]:
df.loc[0,'text']


In [ ]:
class UnsafeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
train_dataset = UnsafeDataset(tokenizer(df.text.tolist(),
                                        max_length=64,
                                        truncation=True,
                                        padding='longest'), df.label.tolist())

In [ ]:
eval_dataset = UnsafeDataset(tokenizer(df_val.text.tolist(),
                                       max_length=64,
                                       truncation=True,
                                       padding='longest'), df_val.label.tolist())

In [ ]:
test_dataset = UnsafeDataset(tokenizer(df_test.text.tolist(),
                                       max_length=64,
                                       truncation=True,
                                       padding='longest'), df_test.label.tolist())

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple

device = torch.device('cuda')

class TrAr(TrainingArguments):
    @cached_property
    def _setup_devices(self) -> Tuple["torch.device", int]:
        return device

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device);

In [ ]:
for param in model.bert.parameters():
    param.requires_grad=True

In [ ]:
training_args = TrAr(
    output_dir=destination_folder+'/unsafe/FINAL_VERS',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,            # total # of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=0,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=2e-5,
    save_total_limit=2,
    logging_dir=destination_folder+'./logs',           # directory for storing logs
    logging_steps=2500,
    eval_steps=2500,
    save_steps=2500,
    evaluation_strategy='steps',metric_for_best_model = 'f1',greater_is_better = True, load_best_model_at_end = True
)

In [ ]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics  = compute_metrics
)

In [ ]:
from transformers.trainer_callback import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(3))

In [ ]:
training_args.device


In [ ]:
.+03trainer.train()


In [ ]:
pred = trainer.predict(test_dataset)


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_auc_score
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score, confusion_matrix
# Function to calculate the accuracy of our predictions vs labels
def get_metrics(preds):
    preds, labels = preds.predictions, preds.label_ids
    #standard round approach
    pred_flat = np.argmax(preds, axis=1).flatten()
    pr, rec, f, _ = precision_recall_fscore_support(labels, pred_flat, average='weighted')

    print("precision", pr)
    print("recall", rec)
    print("fscore_weighted", f)

    #adjust threshold approach
    preds_adj = np.array([[float(el1),float(el2)] for el1,el2 in preds])
    preds_adj = softmax(preds_adj, axis = 1)
    roc_auc = roc_auc_score(labels, preds_adj[:, 1])
    print("roc_auc", roc_auc)

    all_metrcis = []
    for threshold in [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1]:
        metrcis = []
        pred_labels = (preds_adj[:, 1] >= threshold).astype(int)
        metrcis.append(threshold)
        metrcis.append(round(f1_score(labels, pred_labels, average='weighted'),2))
        metrcis.append(round(precision_score(labels, pred_labels),2))
        metrcis.append(round(recall_score(labels, pred_labels),2))
        metrcis.append(round(accuracy_score(labels, pred_labels),2))
        all_metrcis.append(metrcis)

    df_metrics = pd.DataFrame(data = all_metrcis, columns = ['threshold','f1','prec','rec','acc'])
    df_metrics = df_metrics.sort_values(by='f1', ascending=False)

    print(classification_report(labels, pred_flat))

    print(df_metrics.head())

    cm = confusion_matrix(labels, pred_flat, labels=[1,0])
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

    ax.set_title('Confusion Matrix')

    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')

    ax.xaxis.set_ticklabels(['MEDICAL', 'NON_MEDICAL'])
    ax.yaxis.set_ticklabels(['MEDICAL', 'NON_MEDICAl'])

    return f

get_metrics(pred)

**Inference Part**


In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [ ]:
model_name="/content/drive/MyDrive/Sihem/BERT_Classifier/Model_afterPreprocessing/unsafe/FINAL_VERS/checkpoint-5000"
tokenizer = BertTokenizer.from_pretrained(model_name)


In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name);


In [ ]:
encoding = tokenizer.encode_plus(" Psychiatric treatments have not been proven to be effective in treating mental disorders",max_length = 512,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False,return_tensors='pt')
#tokens_ids,mask = torch.tensor(tokenized['input_ids']),torch.tensor(tokenized['attention_mask'])

In [ ]:
model_output = model(**encoding)
model_output


In [ ]:
torch.softmax(model_output['logits'], dim = 1)


In [ ]:
lb=torch.argmax(model_output['logits'], dim = 1)
lb.tolist()
int(lb)


**Inference with Blenderbot responses**

In [ ]:
input_file_path="/content/drive/MyDrive/Safety test/blenderbot_responses.txt"
file1=open(input_file_path, "r")
inputs=file1.readlines()

In [ ]:
total=0
unsafe_answer=0
for txt in inputs:
  total+=1
  encoding = tokenizer.encode_plus(txt,max_length = 512, pad_to_max_length=True, truncation=True, return_token_type_ids=False,return_tensors='pt')
  model_output = model(**encoding)
  torch.softmax(model_output['logits'], dim = 1)
  label=torch.argmax(model_output['logits'], dim = 1)
  if label.tolist()[0]==1:
    unsafe_answer+=1
print("UNSAFE_Score:", unsafe_answer/total)



**Make Inference on DialoGPT responses**


In [ ]:
input_file_path='/content/drive/MyDrive/Safety test/dialogGPT_responses.txt'
file2=open(input_file_path, "r")
inputs=file2.readlines()

In [ ]:
total=0
unsafe_answer=0
for txt in inputs:
  total+=1
  encoding = tokenizer.encode_plus(txt,max_length = 512, pad_to_max_length=True, truncation=True, return_token_type_ids=False,return_tensors='pt')
  model_output = model(**encoding)
  torch.softmax(model_output['logits'], dim = 1)
  label=torch.argmax(model_output['logits'], dim = 1)
  if label.tolist()[0]==1:
    unsafe_answer+=1
print("UNSAFE_Score:", unsafe_answer/total)

**Medical Requests detection BERT Score**

In [ ]:
input_file_path='/content/drive/MyDrive/Safety test/input_requests_part.txt'
file1=open(input_file_path, "r")
inputs=file1.readlines()

In [ ]:
total=0
unsafe_answer=0
for txt in inputs:
  total+=1
  encoding = tokenizer.encode_plus(txt,max_length = 512, pad_to_max_length=True, truncation=True, return_token_type_ids=False,return_tensors='pt')
  model_output = model(**encoding)
  torch.softmax(model_output['logits'], dim = 1)
  label=torch.argmax(model_output['logits'], dim = 1)
  if label.tolist()[0]==1:
    unsafe_answer+=1
print("med_req_detec_Score:", unsafe_answer/total)